# Rotation Matrices and Direction Cosine Matrices

This notebook explores the mathematical foundations of rotation matrices and direction cosine matrices (DCMs) in three-dimensional space. We'll use both numerical calculations with NumPy and symbolic mathematics with SymPy to understand these important concepts in spatial transformations.

## Key Topics

- Elementary rotation matrices for rotations about principal axes
- Composition of rotations using sequential rotation sequences
- Symbolic representation of rotation matrices with LaTeX
- Euler angles and their relation to DCMs

## Setup and Imports

Let's start by importing the libraries we'll need:

In [1]:
import numpy as np
import sympy as sp
from IPython.display import Math, display

sp.init_printing(use_latex=True)

## Elementary Rotation Matrices

Rotation matrices represent rotations in 3D space. The three elementary rotation matrices represent rotations around the principal axes:

### Rotation about the x-axis (Roll, axis=1)

$$R_x(\phi) = \begin{bmatrix} 
1 & 0 & 0 \\
0 & \cos\phi & -\sin\phi \\
0 & \sin\phi & \cos\phi
\end{bmatrix}$$

### Rotation about the y-axis (Pitch, axis=2)

$$R_y(\theta) = \begin{bmatrix} 
\cos\theta & 0 & \sin\theta \\
0 & 1 & 0 \\
-\sin\theta & 0 & \cos\theta
\end{bmatrix}$$

### Rotation about the z-axis (Yaw, axis=3)

$$R_z(\psi) = \begin{bmatrix} 
\cos\psi & -\sin\psi & 0 \\
\sin\psi & \cos\psi & 0 \\
0 & 0 & 1
\end{bmatrix}$$

In [2]:
def create_rotation_matrix(axis, angle):
    c = np.cos(angle)
    s = np.sin(angle)
    if axis == 1:  # X-axis rotation
        return np.array([
            [1, 0, 0],
            [0, c, -s],
            [0, s, c]
        ])
    elif axis == 2:  # Y-axis rotation
        return np.array([
            [c, 0, s],
            [0, 1, 0],
            [-s, 0, c]
        ])
    elif axis == 3:  # Z-axis rotation
        return np.array([
            [c, -s, 0],
            [s, c, 0],
            [0, 0, 1]
        ])
    else:
        raise ValueError("Axis must be 1, 2, or 3")

### Examples of Rotation Matrices

Let's calculate some examples of rotation matrices for different angles:

## Symbolic Representation of Rotation Matrices

We can use SymPy to derive symbolic expressions for rotation matrices and direction cosine matrices (DCMs).

In [3]:
def get_symbolic_dcm(sequence):
    phi, theta, psi = sp.symbols('\\phi \\theta \\psi')
    angle_map = {1: phi, 2: theta, 3: psi}
    angles= [angle_map[axis] for axis in sequence]
    def create_symbolic_rotation(axis, angle):
        c = sp.cos(angle)
        s = sp.sin(angle)
        if axis == 1:
            return sp.Matrix([
                [1, 0, 0],
                [0, c, -s],
                [0, s, c]
            ])
        elif axis == 2:
            return sp.Matrix([
                [c, 0, s],
                [0, 1, 0],
                [-s, 0, c]
            ])
        elif axis == 3:
            return sp.Matrix([
                [c, -s, 0],
                [s, c, 0],
                [0, 0, 1]
            ])
    dcm = sp.eye(3)
    for axis, angle in zip(sequence, angles):
        rot_matrix = create_symbolic_rotation(axis, angle)
        dcm = dcm * rot_matrix
    return dcm

### Common Rotation Sequences

Let's examine some common rotation sequences and their symbolic representations:

1. **Aircraft Euler Angles (3-2-1 sequence)**: Yaw (ψ) → Pitch (θ) → Roll (φ)
2. **Classical Euler Angles (3-1-3 sequence)**: Precession (ψ) → Nutation (θ) → Spin (φ)

In [4]:
def display_symbolic_dcm(sequence, description):
    """Display symbolic DCM with LaTeX formatting"""
    dcm = get_symbolic_dcm(sequence)
    dcm_inverse = dcm.transpose()
    
    
    axis_names = {1: "X", 2: "Y", 3: "Z"}
    rotation_sequence = '-'.join([axis_names[axis] for axis in sequence])
    
    print(f"{description} (Sequence: {rotation_sequence})")
    display(Math(sp.latex(dcm)))
    print(f"Inverse DCM :")
    display(Math(sp.latex(dcm_inverse)))

In [5]:
aircraft_dcm = display_symbolic_dcm([3, 2, 1], "Aircraft Euler Angles (Yaw-Pitch-Roll)")

Aircraft Euler Angles (Yaw-Pitch-Roll) (Sequence: Z-Y-X)


<IPython.core.display.Math object>

Inverse DCM :


<IPython.core.display.Math object>

## Numerical Calculations of Direction Cosine Matrices

Now let's create a function to calculate DCMs numerically for any rotation sequence:

In [6]:
def euler_to_dcm(sequence, angles):
    if len(sequence) != len(angles):
        raise ValueError("Number of axes must match number of angles")
    
    dcm = np.eye(3)
    
    for axis, angle in zip(sequence, angles):
        rotation_matrix = create_rotation_matrix(axis, angle)
        dcm = np.dot(dcm, rotation_matrix)
    
    return dcm

### Example: Computing DCMs for Aircraft Motion

Let's compute a DCM for an aircraft with specific Euler angles (yaw, pitch, roll):

In [7]:
def print_matrix(matrix, precision=6):

    rows, cols = matrix.shape
    
    for i in range(rows):
        print("  [", end="")
        for j in range(cols):
            value = matrix[i, j]
            formatted_value = f"{value:{precision+4}.{precision}f}"
            print(f"{formatted_value}", end="")
            if j < cols - 1:
                print(", ", end="")
        print("]" + ("" if i < rows - 1 else ""))
    

In [8]:
# Example: Aircraft with yaw=30°, pitch=15°, roll=10°
sequence = [3, 2, 1]  # Yaw-Pitch-Roll (Z-Y-X)
angles_deg = [30, 15, 10]  
angles_rad = [np.radians(angle) for angle in angles_deg]

print("Aircraft Euler Angles:")
print(f"Yaw (ψ): {angles_deg[0]}°")
print(f"Pitch (θ): {angles_deg[1]}°")
print(f"Roll (φ): {angles_deg[2]}°\n")

dcm = euler_to_dcm(sequence, angles_rad)
print("Direction Cosine Matrix (DCM):")
print_matrix(dcm)

Aircraft Euler Angles:
Yaw (ψ): 30°
Pitch (θ): 15°
Roll (φ): 10°

Direction Cosine Matrix (DCM):
  [  0.836516,  -0.453482,   0.307563]
  [  0.482963,   0.875340,  -0.022940]
  [ -0.258819,   0.167731,   0.951251]


In [ ]:
sequences = [
    [3, 2, 1],  # Z-Y-X
    [2, 3, 1],  # Y-Z-X
    [2, 1, 3],  # Y-X-Z
    [1, 3, 2],  # X-Z-Y
    [1, 2, 3],  # X-Y-Z
    [3, 1, 2]   # Z-X-Y
]

angles_deg = [5, 3, 1]
angles_rad = [np.radians(angle) for angle in angles_deg]
angle_names = {1: "φ (phi)", 2: "θ (theta)", 3: "ψ (psi)"}
axis_names = {1: "X", 2: "Y", 3: "Z"}

print("Numerical Results:\n")

for sequence in sequences:
    axis_names = {1: "X", 2: "Y", 3: "Z"}
    rotation_sequence = '-'.join([axis_names[axis] for axis in sequence])
    angle_description = ", ".join([f"{angle_names[axis]}: {angles_deg[i]}°" for i, axis in enumerate(sequence)])
    print(f"Angles: {angle_description}")
    dcm = euler_to_dcm(sequence, angles_rad)
    print_matrix(dcm)
    print(f"\n {'-'*60} \n")

Numerical Results:
Angles: ψ (psi): 5°, θ (theta): 3°, φ (phi): 1°
  [  0.994829,  -0.086233,   0.053650]
  [  0.087036,   0.996123,  -0.012825]
  [ -0.052336,   0.017428,   0.998477]

 ------------------------------------------------------------ 

Angles: θ (theta): 5°, ψ (psi): 3°, φ (phi): 1°
  [  0.994829,  -0.050608,   0.088052]
  [  0.052336,   0.998477,  -0.017428]
  [ -0.087036,   0.021947,   0.995963]

 ------------------------------------------------------------ 

Angles: θ (theta): 5°, φ (phi): 3°, ψ (psi): 1°
  [  0.996123,  -0.012825,   0.087036]
  [  0.017428,   0.998477,  -0.052336]
  [ -0.086233,   0.053650,   0.994829]

 ------------------------------------------------------------ 

Angles: φ (phi): 5°, ψ (psi): 3°, θ (theta): 1°
  [  0.998477,  -0.052336,   0.017428]
  [  0.053650,   0.994829,  -0.086233]
  [ -0.012825,   0.087036,   0.996123]

 ------------------------------------------------------------ 

Angles: φ (phi): 5°, θ (theta): 3°, ψ (psi): 1°
  [  0.998477